# Interacting with Assets in Markets API

This notebook provides a comprehensive guide on how to interact with different types of assets using the VAM API.
We will cover:
- Querying different asset types
- Filtering assets by execution venue
- Working with different asset types like spot, futures, and equity
- Performing batch operations



We will demonstrate how to query assets across these execution venues.


In [1]:
import os
from pathlib import Path

# Save the original working directory (only once)
try:
    original_wd
except NameError:
    original_wd = os.getcwd()

# Compute the target directory: two levels up from the original working directory
# For example, if original_wd is /Users/username/project/notebooks,
# then target_dir becomes /Users/username
target_dir = Path(original_wd).parents[2]

# Change the working directory to the target directory
os.chdir(target_dir)
print("Working directory set to:", os.getcwd())

Working directory set to: /home/jose/code/MainSequenceClientSide/mainsequence-sdk


In [2]:

import dotenv
env_loaded=dotenv.load_dotenv('.env')
print(f"Env file loaded: {env_loaded}")
import mainsequence 
from mainsequence.client import Asset, AssetFutureUSDM, AssetCurrencyPair,AssetCategory
from mainsequence.client import MARKETS_CONSTANTS

# Define execution venue symbols
BINANCE_EV = MARKETS_CONSTANTS.BINANCE_EV_SYMBOL
BINANCE_FUTURES_EV = MARKETS_CONSTANTS.BINANCE_FUTURES_EV_SYMBOL
ALPACA_EV = MARKETS_CONSTANTS.ALPACA_EV_SYMBOL

Env file loaded: True


2025-05-23T07:58:44.171852Z [debug    ] Getting Auth Headers ASSETS_ORM application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:39 in refresh_headers())
2025-05-23T07:58:44.391345Z [debug    ] Getting Auth Headers ASSETS_ORM application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:39 in refresh_headers())
2025-05-23T07:58:44.788348Z [debug    ] took 0.3601 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/pods/projects/get_user_default_project/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:95 in make_request())
2025-05-23T07:58:44.789438Z [info     ] Set remote data source to orm_class='DataSource' id=1 display_name='Default TS GCP' organization=1 class_type='timescale_db' status='AVAILABLE' extra_arguments=None application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at models_tdag.py:2114 in set_remote_db())
2025-05-23T07:58:44.863152Z [debug    ] Getting Auth Headers ASSE

## Registering new assets

Main Sequence Assets are closely aligned with the Open FIGI asset master list available at https://www.openfigi.com/. Each asset on the Main Sequence platform shares identical properties with its corresponding asset listed in Open FIGI. The primary difference on the Main Sequence platform is that each asset also includes a unique identifier determined by the execution venue.

It is important to note that an execution venue is not always synonymous with an exchange. The execution venue refers to the entity or platform where the account holding the asset is maintained. For example, Interactive Brokers could be an execution venue. This means that within an Interactive Brokers account, one might hold NVDA shares listed on a specific exchange with a FIGI from https://www.openfigi.com/search?marketSector=All&searchTerms=NVDA. However, this holding must be differentiated from NVDA holdings maintained in an account at another execution venue, such as Alpaca. Although the FIGIs remain the same, the unique identifiers differ. On the Main Sequence platform, the unique identifier combines the asset ticker, FIGI, and the execution venue symbol.

Given the extensive and continuously growing number of assets, not all are automatically registered in the Main Sequence platform. Nevertheless, users have the option to manually register assets at a specific execution venue using our client library.

In [3]:
# we will register the following figi BBG014T46NC0 which corresponds  to NVDA trading in the Toronto Stock Exchange and BBG00GQ6S7X0 corresponds to NVDA in Vienna stock exchange
figi_to_register=["BBG014T46NC0","BBG00GQ6S7X0"]
for f in figi_to_register:
    Asset.register_figi_as_asset_in_main_sequence_venue(figi=f,
                                          timeout=100000
                                          
                                          )
    registered_asset=Asset.get(figi=f,execution_venue__symbol=MARKETS_CONSTANTS.MAIN_SEQUENCE_EV)
    registered_asset.pretty_print()


2025-05-23T07:58:45.697329Z [debug    ] took 0.4337 seconds. Requesting POST from http://127.0.0.1:8000/orm/api/assets/asset/register_figi_as_asset_in_main_sequence_venue/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:95 in make_request())
2025-05-23T07:58:46.132327Z [debug    ] took 0.4322 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/assets/asset/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:95 in make_request())


Property                  | Value                                                                        
---------------------------------------------------------------------------------------------------------
orm_class                 | Asset
id                        | 32799
can_trade                 | False
execution_venue           | orm_class='ExecutionVenue' id=6 symbol='ms_share_class' name='Main Sequence '
delisted_datetime         | None
unique_identifier         | NVDA_ms_share_class_BBG014T46NC0
real_figi                 | True
is_custom_by_organization | False
figi                      | BBG014T46NC0
composite                 | BBG014T46N03
ticker                    | NVDA
security_type             | Canadian DR
security_type_2           | Depositary Receipt
security_market_sector    | Equity
share_class               | BBG014T46P17
exchange_code             | TX
name                      | NVIDIA CORP-CDR
main_sequence_share_class | NztvcOeykeuQ
isin                     

2025-05-23T07:58:46.561880Z [debug    ] took 0.4281 seconds. Requesting POST from http://127.0.0.1:8000/orm/api/assets/asset/register_figi_as_asset_in_main_sequence_venue/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:95 in make_request())
2025-05-23T07:58:47.003751Z [debug    ] took 0.4408 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/assets/asset/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:95 in make_request())


Property                  | Value                                                                        
---------------------------------------------------------------------------------------------------------
orm_class                 | Asset
id                        | 32800
can_trade                 | False
execution_venue           | orm_class='ExecutionVenue' id=6 symbol='ms_share_class' name='Main Sequence '
delisted_datetime         | None
unique_identifier         | NVDA_ms_share_class_BBG00GQ6S7X0
real_figi                 | True
is_custom_by_organization | False
figi                      | BBG00GQ6S7X0
composite                 | BBG00GQ6S7W1
ticker                    | NVDA
security_type             | Common Stock
security_type_2           | Common Stock
security_market_sector    | Equity
share_class               | BBG001S5TZJ6
exchange_code             | AV
name                      | NVIDIA CORP
main_sequence_share_class | m8qqW6CbSUAo
isin                      | None


It is important to clarify what happens when we register an asset in the Main Sequence EV:

1) An asset will always be created with a synthetic Fiji, inferred from a combination of ticker, exchange_code, security_type, market_sector, and security_type_2, with exchange_code set as None.

2) The main_sequence_share_class is inferred from the combination of ticker, security_type, market_sector, and security_type_2. The purpose of this class is to enable working with the same asset across both backtesting and analysis environments without being dependent on a specific exchange. For instance, when referencing a fundamentals table, we want an NVDA asset representation that remains independent of any specific exchange_code.

For this reason, in the example below, we have five assets with the ticker NVDA. One asset trades through Alpaca with a general US reference as its execution_venue and shares the same main_sequence_share_class as two other assets in the Main Sequence EV—one with no exchange code and one trading on the Vienna Stock Exchange.

In [4]:
assets_in_venue=Asset.filter(ticker="NVDA")

print(len(assets_in_venue))

for a in assets_in_venue:
    print(a.ticker,a.main_sequence_share_class,a.unique_identifier,a.exchange_code,a.execution_venue.symbol)
   



2025-05-23T07:58:47.537270Z [debug    ] took 0.4660 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/assets/asset/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:95 in make_request())


5
NVDA m8qqW6CbSUAo NVDA_ms_share_class_m8qqW6CbSUAo None ms_share_class
NVDA m8qqW6CbSUAo NVDA_alpaca_BBG000BBJQV0 US alpaca
NVDA NztvcOeykeuQ NVDA_ms_share_class_l7bJT26orL54 None ms_share_class
NVDA NztvcOeykeuQ NVDA_ms_share_class_BBG014T46NC0 TX ms_share_class
NVDA m8qqW6CbSUAo NVDA_ms_share_class_BBG00GQ6S7X0 AV ms_share_class


### Registering Assets by ISIN





In [5]:
created_asset=Asset.get_or_register_figi_from_isin_as_asset_in_main_sequence_venue(isin="US46434V4077",exchange_code="MF", #Bolsa Mexican de Valores
                                          timeout=100000

                                          )
created_asset.pretty_print()

2025-05-23T07:58:48.011636Z [debug    ] took 0.4531 seconds. Requesting POST from http://127.0.0.1:8000/orm/api/assets/asset/get_or_register_figi_from_isin_as_asset_in_main_sequence_venue/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:95 in make_request())


Property                  | Value                                                                        
---------------------------------------------------------------------------------------------------------
orm_class                 | Asset
id                        | 32809
can_trade                 | False
execution_venue           | orm_class='ExecutionVenue' id=6 symbol='ms_share_class' name='Main Sequence '
delisted_datetime         | None
unique_identifier         | SHYG*_ms_share_class_BBG00B0GXPL4
real_figi                 | True
is_custom_by_organization | False
figi                      | BBG00B0GXPL4
composite                 | BBG00B0GXPK5
ticker                    | SHYG*
security_type             | ETP
security_type_2           | Mutual Fund
security_market_sector    | Equity
share_class               | BBG005DW06B7
exchange_code             | MF
name                      | ISHARES 0-5 YR HY CORP BOND
main_sequence_share_class | rr9lXpSVRuqt
isin                      

### Registering a custom Asset
Occasionally, you may encounter an asset that isn’t yet in the OpenFIGI database. In these edge cases, the Main Sequence platform lets you register custom assets on our execution venue. Although this can be useful, we strongly encourage you to request that both FIGIs and ISINs be added to OpenFIGI. If an instrument already has an ISIN but no FIGI, try to match its attributes—such as issuer, currency, maturity date, etc.—as closely as possible when creating the custom record.

**Important**

When you register a non-FIGI asset, the platform will infer the FIGI by concatenating its ticker, exchange_code, security_type, market_sector, and security_type_2. Likewise, the main_sequence_share_class is inferred from ticker, security_type, market_sector, and security_type_2. Finally, the asset’s unique identifier is always formatted as `f"{asset.ticker}_{asset.execution_venue.symbol}_{asset.figi}_{request.user.organization.uuid}"`

Please note that Security Type, Security Type 2, Market Sector, and Exchange Code must appear on the allowed lists (see https://www.openfigi.com/api/documentation); if they do not, leave them as None.




In [6]:
new_registered_custom_asset=Asset.get_or_register_custom_asset_in_main_sequence_venue(ticker="PBD",
                                                                                      name="PBD Custom Asset",
                                                                                    security_type=None,           # e.g. one of OpenFIGI’s Security Type values
                                                                                    security_type_2=None,       # e.g. OpenFIGI SecurityType2
                                                                                    security_market_sector=None,       # e.g. OpenFIGI Market Sector
                                                                                    isin="US1234567890",                      # your ISIN
                                                                                    exchange_code=None
                                                                                      )
new_registered_custom_asset.pretty_print()

2025-05-23T07:58:53.998794Z [debug    ] took 5.9786 seconds. Requesting POST from http://127.0.0.1:8000/orm/api/assets/asset/get_or_register_custom_asset_in_main_sequence_venue/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:95 in make_request())


Property                  | Value                                                                        
---------------------------------------------------------------------------------------------------------
orm_class                 | Asset
id                        | 32821
can_trade                 | False
execution_venue           | orm_class='ExecutionVenue' id=6 symbol='ms_share_class' name='Main Sequence '
delisted_datetime         | None
unique_identifier         | PBD_ms_share_class_ZU8QZaIUMTec
real_figi                 | False
is_custom_by_organization | True
figi                      | ZU8QZaIUMTec
composite                 | None
ticker                    | PBD
security_type             | None
security_type_2           | None
security_market_sector    | None
share_class               | None
exchange_code             | None
name                      | PBD Custom Asset
main_sequence_share_class | 5x49DhAHV2dF
isin                      | US1234567890


## Optimal Asset Queries

There are many assets, and there will always be more. We recommend narrowing your search so that the expected results are in the hundreds. If you need more assets, it is best to first create a category that encompasses those assets and then query by that category.

A good way to start narrowing asset searches is by filtering on the execution venue and some FIGI properties. For example, if you want to retrieve the equivalent assets from one category on another exchange, you could do something like this:



In [7]:
top_100_cryptos = AssetCategory.get(unique_identifier="top_100_crypto_market_cap")

# Switch to Binance categories
spot_assets = Asset.filter(id__in=top_100_cryptos.assets)

# Get them through the main sequence FIGI class and exchange
binance_currency_pairs = AssetCurrencyPair.filter(
    base_asset__main_sequence_share_class__in=[
        a.main_sequence_share_class for a in spot_assets
    ],
    execution_venue__symbol=MARKETS_CONSTANTS.BINANCE_EV_SYMBOL,
    quote_asset__ticker="USDT",
    include_base_quote_detail=False
)



2025-05-23T07:58:54.505030Z [debug    ] took 0.4972 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/assets/asset-category/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:95 in make_request())
2025-05-23T07:58:55.065547Z [debug    ] took 0.5587 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/assets/asset/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:95 in make_request())
2025-05-23T07:58:55.707416Z [debug    ] took 0.6393 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/assets/asset_currency_pair/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:95 in make_request())


## Filtering Assets by Type

Different asset types have distinct properties, which can help you extend your filters. It's important to note that all asset types inherit from the Asset class. Therefore, you can always use the Asset class to access all assets and their general properties.

In [8]:
# Filtering Simple Assets
simple_assets = Asset.filter(ticker="BTCUSDT")
print(f"Total Spot Crypto Assets on Binance: {len(simple_assets)}")

# Filtering Futures
cash_equity_assets = AssetCurrencyPair.filter(ticker="BTCUSDT")
print(f"Total Assets CurrencyPair Assets : {len(cash_equity_assets)}")

# Filtering  Futures USDM Assets
futures_assets= AssetFutureUSDM.filter(ticker="BTCUSDT")
print(f"Total Futures Assets : {len(futures_assets)}")




2025-05-23T07:58:56.261161Z [debug    ] took 0.5423 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/assets/asset/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:95 in make_request())


Total Spot Crypto Assets on Binance: 5


2025-05-23T07:58:56.910348Z [debug    ] took 0.6479 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/assets/asset_currency_pair/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:95 in make_request())


Total Assets CurrencyPair Assets : 3


2025-05-23T07:58:57.575829Z [debug    ] took 0.6642 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/assets/asset_future_usdm/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:95 in make_request())


Total Futures Assets : 2


## Using `filter_with_asset_class`

The `filter_with_asset_class` method is an extended version of the `filter` method.
It ensures that each returned asset is an instance of its correct class 

### When to Use?
- When querying multiple asset types and needing them returned with the correct class.
- When working with assets across execution venues and requiring proper type differentiation.

### Example 1: Query all asset types for a symbol


In [9]:
# Query Binance Spot assets with correct asset classes
all_btc_assets = Asset.filter_with_asset_class(ticker="BTCUSDT")

# Displaying asset information
for asset in all_btc_assets:
    print(f"Asset: {asset.ticker},  Class: {type(asset).__name__}")


2025-05-23T07:58:58.782946Z [debug    ] took 1.1939 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/assets/asset/list_with_asset_class/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:95 in make_request())


Asset: BTCUSDT,  Class: AssetCurrencyPair
Asset: BTCUSDT,  Class: AssetFutureUSDM
Asset: BTCUSDT,  Class: AssetFutureUSDM
Asset: BTCUSDT,  Class: AssetCurrencyPair
Asset: BTCUSDT,  Class: AssetCurrencyPair


### Example 2: Using FIGI Information to Narrow Down Results

Now imagine we only want currency pairs instead of futures.
 In this case, we can use FIGI details to identify the asset classes we are specifically interested in.



In [10]:
all_curreny_pairs_bitcoin= Asset.filter_with_asset_class(ticker="BTCUSDT",
security_market_sector=MARKETS_CONSTANTS.FIGI_MARKET_SECTOR_CURNCY,
security_type=MARKETS_CONSTANTS.FIGI_SECURITY_TYPE_CRYPTO
)

print(all_curreny_pairs_bitcoin)

2025-05-23T07:58:59.625521Z [debug    ] took 0.8293 seconds. Requesting GET from http://127.0.0.1:8000/orm/api/assets/asset/list_with_asset_class/ application_name=ms-sdk data_source_id=1 job_run_id=None project_id=1 (at utils.py:95 in make_request())


[AssetCurrencyPair: BTCUSDT_ms_share_class_MSyq4PRl58KI, AssetCurrencyPair: BTCUSDT_bnce_xF5EceMkk2S4, AssetCurrencyPair: BTCUSDT_bnf8_29EIYV7huAXC]
